# Bias Variance

![title](ml_bias_variance.png)

Мы не будем выписывать строгие формулы, но попробуем объяснить идею этих понятий.

Пусть у нас есть алгоритм обучения, который по данным может создать модель.

Ошибка этих моделей может быть разложена на три части:
* **Noise** – шум данных, не предсказуем, теоретический минимум ошибки
* **Bias** – смещение, на сколько хорошо работает средний алгоритм. Средний алгоритм это "возьмём случайные данные, обучим алгоритм, сделаем предсказания", **Bias** – это ошибка средних предсказаний.
* **Variance** – разброс, на сколько устойчиво работает алгоритм. Опять же "возьмём случайные данные, обучим алгоритм, сделаем предсказания", **Variance** – это разрос этих предсказаний.

# Бустинг и Бэггинг в терминах Bias и Variance

Как вы думаете на какую составляющую Бустинг и Бэггинг влияют, а на какую нет?

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('HR.csv')

target = 'left'
features = [c for c in data if c != target]
print(features)

X, y = np.array(data[features]), np.array(data[target])

['last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
def sample_model(seed, model):
    random_gen = np.random.RandomState(seed)
    indices = random_gen.choice(len(y_train), size=len(y_train), replace=True)
    model.fit(X_train[indices, :], y_train[indices])
    return model

def estimate_bias_variance(model, iters_count=100):
    y_preds = []
    for seed in range(iters_count):
        model = sample_model(seed, model)
        y_preds.append(model.predict(X_test))
    y_preds = np.array(y_preds)
    
    print('Bias:', np.mean((y_test - y_preds.mean(axis=0)) ** 2))
    print('Variance:', y_preds.std(axis=0).mean())

In [6]:
estimate_bias_variance(LinearRegression())

Bias: 0.221637100837659
Variance: 0.010670490556716294


In [7]:
estimate_bias_variance(DecisionTreeRegressor(max_depth=5))

Bias: 0.16922565195887232
Variance: 0.04012902050067083


In [8]:
estimate_bias_variance(DecisionTreeRegressor(max_depth=10))

Bias: 0.16626637714593603
Variance: 0.11381880012416418


In [9]:
estimate_bias_variance(DecisionTreeRegressor(max_depth=15))

Bias: 0.17324908974181663
Variance: 0.21381985964171496


In [10]:
estimate_bias_variance(DecisionTreeRegressor(max_depth=None))

Bias: 0.19912198252892632
Variance: 0.32234001952881103


In [11]:
estimate_bias_variance(RandomForestRegressor(n_estimators=1, random_state=42))

Bias: 0.18753342962384387
Variance: 0.3540485777541925


In [12]:
estimate_bias_variance(RandomForestRegressor(n_estimators=10, random_state=42))

Bias: 0.18697729299866533
Variance: 0.17074998023991847


In [13]:
estimate_bias_variance(RandomForestRegressor(n_estimators=50, random_state=42))

Bias: 0.18693070199034895
Variance: 0.14097370973547677


In [16]:
estimate_bias_variance(XGBRegressor(n_estimators=1, max_depth=20))

Bias: 0.23451946472392102
Variance: 0.022801744


In [17]:
estimate_bias_variance(XGBRegressor(n_estimators=1, max_depth=10))

Bias: 0.2341107071773297
Variance: 0.010385236


In [18]:
estimate_bias_variance(XGBRegressor(n_estimators=1, max_depth=5))

Bias: 0.23507719247660752
Variance: 0.003943091


In [19]:
estimate_bias_variance(XGBRegressor(n_estimators=10, max_depth=5))

Bias: 0.1783928390733272
Variance: 0.018791953


In [20]:
estimate_bias_variance(XGBRegressor(n_estimators=100, max_depth=5))

Bias: 0.1666821804160293
Variance: 0.056974187


In [21]:
estimate_bias_variance(CatBoostRegressor(n_estimators=1, max_depth=6, verbose=False))

Bias: 0.332715163680473
Variance: 0.0007108716806930628


In [22]:
estimate_bias_variance(CatBoostRegressor(n_estimators=10, max_depth=6, verbose=False))

Bias: 0.26696809804301386
Variance: 0.0054989897738102495


In [23]:
estimate_bias_variance(CatBoostRegressor(n_estimators=100, max_depth=6, verbose=False))

Bias: 0.17119445116377977
Variance: 0.019270308684423625
